# Preprocessing CyGNSS data

Data is downloaded from NASA EarthCloud as described in the `APIs` notebook. For the expected format for CyGNSSnet, additional preprocessing steps are necessary.

In [1]:
import os
import sys
sys.path.append('../externals/gfz_cygnss/') 

In [2]:
# !pip install tenacity

In [3]:
from gfz_202003.preprocessing import preprocess as prep

In [4]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import seaborn as sns

import datetime
import xarray as xr

In [5]:
import cdsapi

## Download raw CyGNSS data

The CyGNSSnet preprocessing routine expects the raw data files ordered as 

> `$raw_data_dir/<year>/<day-of-year>/cyg*.nc`

Data is always downloaded for one full day for all spacecraft, generating 8 `netcdf` files per day of observations. Below is a routine to specify a date range, followed by downloading the corresponding data and storing it in the appropriate subfolders.

In [6]:
raw_data_root = '/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/raw_data'
dev_data_root = '/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/dev_data'

Select a test day and prepared the input parameters for the provided download script

In [7]:
year  = 2021
month = 3
day   = 17

Downloaded target directory in the expected format `year/day-of-year`

In [8]:
raw_data_sub = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%Y/%j")

raw_data_dir = os.path.join(raw_data_root, raw_data_sub)

print(raw_data_dir)

/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/raw_data/2021/076


Start and end date of download range in the required format. The end date is midnight the next day, this way only the requested day's data is downloaded.

In [9]:
start_date = datetime.datetime(year, month, day).strftime("%Y-%m-%dT%H:%M:%SZ")
end_date   = datetime.datetime(year, month, day + 1).strftime("%Y-%m-%dT%H:%M:%SZ")

print(f'--start-date {start_date}')
print(f'--end-date   {end_date}')

--start-date 2021-03-17T00:00:00Z
--end-date   2021-03-18T00:00:00Z


In [13]:
%env PYTHONPATH=/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber
!python /home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber/subscriber/podaac_data_downloader.py  -c CYGNSS_L1_V3.1 -d $raw_data_dir --start-date $start_date --end-date $end_date

env: PYTHONPATH=/home/harsh/Downloads/DKRZ/MLOps/2022-cygnss-deployment/data-subscriber
[2022-09-12 16:00:57,433] {podaac_data_downloader.py:243} INFO - Found 7 total files to download
[2022-09-12 16:00:59,062] {podaac_access.py:446} WARNING - Computed checksum f11baba7acac4b5b14b3891e83f715c8 does not match expected checksum 10e4ef36d29f030ea7e524f8924389fc
[2022-09-12 16:01:46,860] {podaac_data_downloader.py:276} INFO - 2022-09-12 16:01:46.860919 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg06.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc
[2022-09-12 16:01:48,483] {podaac_access.py:446} WARNING - Computed checksum 9b3100d23550d03cb85056609ecddd5b does not match expected checksum a8851840f3a4bbdc8499ea2f17d5119b
[2022-09-12 16:02:39,804] {podaac_data_downloader.py:276} INFO - 2022-09-12 16:02:39.804552 SUCCESS: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/CYGNSS_L1_V3.1/cyg08.ddmi.s2021061

## Download raw ERA5 data

The preprocessing pipeline requires the ERA5 windspeed labels. Download the raw ERA5 data for the same timespan.

In [10]:
era5_data = os.path.join(raw_data_dir, 'ERA5_windspeed.nc')

In [11]:
cds = cdsapi.Client()

In [12]:
cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': year,
        'month': month,
        'day': day,
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00'
        ],
        'area': [
            40, -180, -40, 180,
        ],
    },
    era5_data)

/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-07 17:16:00,801 INFO Welcome to the CDS
2022-10-07 17:16:00,803 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2022-10-07 17:16:00,893 INFO Request is completed
2022-10-07 17:16:00,895 INFO Downloading https://download-0016.copernicus-climate.e

Result(content_length=44383516,content_type=application/x-netcdf,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data4/adaptor.mars.internal-1665152056.1785328-15145-5-48274a21-4e95-4e14-814a-77896d352d65.nc)

In [13]:
era5_ds = xr.open_dataset(era5_data)
era5_ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 321, time: 24)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 40.0 39.75 39.5 39.25 ... -39.5 -39.75 -40.0
  * time       (time) datetime64[ns] 2021-03-17 ... 2021-03-17T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-07 14:14:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## Annotate raw CyGNSS data with windspeed labels

We need to create the data variables `ERA5_u10` and `ERA5_v10` and attach them to the CyGNSS raw data.

In [14]:
os.listdir(raw_data_dir)

['ERA5_windspeed.nc',
 'cyg04.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg02.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg03.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'CYGNSS_L1_V3.1.citation.txt',
 'cyg01.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg07.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg06.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg08.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc',
 'cyg05.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc']

In [15]:
cygnss_file = os.path.join(raw_data_dir, 'cyg07.ddmi.s20210317-000000-e20210317-235959.l1.power-brcs.a31.d32.nc')

# necessary because lazy loading prohibits overwriting the netcdf files at the end of this section
with xr.open_dataset(cygnss_file) as data:
    cygnss_ds = data.load()
    
#cygnss_ds.close() # close I/O stream
cygnss_ds

<xarray.Dataset>
Dimensions:                          (sample: 172800, ddm: 4, delay: 17, doppler: 11)
Coordinates:
  * sample                           (sample) int32 0 1 2 ... 172798 172799
  * ddm                              (ddm) int8 0 1 2 3
    ddm_timestamp_utc                (sample) datetime64[ns] 2021-03-17T00:00...
    sp_lat                           (sample, ddm) float32 36.74 34.32 ... 3.283
    sp_lon                           (sample, ddm) float32 94.3 87.65 ... 144.0
Dimensions without coordinates: delay, doppler
Data variables: (12/137)
    spacecraft_id                    int16 55
    spacecraft_num                   int8 7
    ddm_source                       int8 2
    ddm_time_type_selector           int8 1
    delay_resolution                 float32 0.2552
    dopp_resolution                  float32 500.0
    ...                               ...
    ERA5_swh                         int64 -9999
    ERA5_shts                        int64 -9999
    ERA5_shww                        int64 -9999
    ERA5_p140121                     int64 -9999
    ERA5_p140124                     int64 -9999
    ERA5_p140127                     int64 -9999
Attributes: (12/66)
    Conventions:                          CF-1.6, ACDD-1.3, ISO-8601
    standard_name_vocabulary:             CF Standard Name Table v30
    project:                              CYGNSS
    featureType:                          trajectory
    summary:                              CYGNSS is a NASA Earth Venture miss...
    program:                              CYGNSS
    ...                                   ...
    geospatial_lon_max:                   359.999E
    time_coverage_resolution:             P0DT0H0M1S
    time_coverage_start:                  2021-03-17T00:00:00.499261765Z
    time_coverage_end:                    2021-03-17T23:59:59.999261678Z
    time_coverage_duration:               P1DT0H0M0S
    NCO:                                  4.4.4

Check units for spacetime coordinates
* Longitude
  * ERA5:   -180 ... 0 ... +180
  * CyGNSS: 0 ... 180 ... 360
* Latitude
  * ERA5 & CyGNSS: -40 ... 0 ... +40
* Timestamp


--> Need to shift the ERA5 longitude coordinate by 180

In [16]:
era5_ds = era5_ds.assign_coords(longitude=era5_ds.coords['longitude'] + 180)
era5_ds

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 321, time: 24)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 40.0 39.75 39.5 39.25 ... -39.5 -39.75 -40.0
  * time       (time) datetime64[ns] 2021-03-17 ... 2021-03-17T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-07 14:14:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
sample_ix = 4821
ddm_ix    = 2

sample_ds = cygnss_ds.sel(sample=sample_ix, ddm=ddm_ix)
sample_ds

sample_sp_lat = sample_ds.sp_lat.values
print(sample_sp_lat)

sample_sp_lon = sample_ds.sp_lon.values
print(sample_sp_lon)

sample_ddm_timestamp_utc = sample_ds.ddm_timestamp_utc.values
print(sample_ddm_timestamp_utc)

-35.858727
225.74704
2021-03-17T00:40:10.999261603


In [18]:
interp_ds = era5_ds.interp(longitude=cygnss_ds.sp_lon, latitude=cygnss_ds.sp_lat, time=cygnss_ds.ddm_timestamp_utc)
interp_ds

/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/home/harsh/anaconda3/envs/cygnss-d/lib/python3.9/site-packages/xarray/core/missing.py:563: FutureWarning: Passi

<xarray.Dataset>
Dimensions:            (sample: 172800, ddm: 4)
Coordinates:
    longitude          (sample, ddm) float32 94.3 87.65 89.29 ... 144.3 144.0
    latitude           (sample, ddm) float32 36.74 34.32 27.98 ... 2.238 3.283
    time               (sample) datetime64[ns] 2021-03-17T00:00:00.499261765 ...
  * sample             (sample) int32 0 1 2 3 4 ... 172796 172797 172798 172799
  * ddm                (ddm) int8 0 1 2 3
    ddm_timestamp_utc  (sample) datetime64[ns] 2021-03-17T00:00:00.499261765 ...
    sp_lat             (sample, ddm) float32 36.74 34.32 27.98 ... 2.238 3.283
    sp_lon             (sample, ddm) float32 94.3 87.65 89.29 ... 144.3 144.0
Data variables:
    u10                (sample, ddm) float64 0.8028 -2.937 -0.2886 ... nan nan
    v10                (sample, ddm) float64 -0.06948 1.576 5.92 ... nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2022-10-07 14:14:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [19]:
cygnss_ds['ERA5_u10'] = interp_ds['u10']
cygnss_ds['ERA5_v10'] = interp_ds['v10']

tmp_attrs = cygnss_ds['ERA5_u10'].attrs
tmp_attrs['long_name'] = cygnss_ds['ERA5_u10'].long_name + ' (interpolated)'
cygnss_ds['ERA5_u10'].attrs = tmp_attrs

tmp_attrs = cygnss_ds['ERA5_v10'].attrs
tmp_attrs['long_name'] = cygnss_ds['ERA5_v10'].long_name + ' (interpolated)'
cygnss_ds['ERA5_v10'].attrs = tmp_attrs

In [20]:
cygnss_ds = cygnss_ds.drop_vars(['longitude', 'latitude', 'time'])

TODO: drop the variables that are not used later on and apply the quality filter here

Overwrite the dataset on disk

In [21]:
cygnss_ds.to_netcdf(cygnss_file)

## Check raw data

In [22]:
from importlib import reload
reload(prep)
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, cygnss_file))

raw_ds

ValueError: cannot swap from dimension 'c' because it is not an existing dimension

In [23]:
filtered_ds = prep.apply_quality_filter(raw_ds, is_ml_ops=True)
filtered_ds

NameError: name 'raw_ds' is not defined

In [ ]:
os.listdir('/work/ka1176/shared_data/2020-03/raw_data/2021/014/')

In [ ]:
old_raw_ds = xr.open_mfdataset('/work/ka1176/shared_data/2020-03/raw_data/2022/168/cyg06.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc')
old_raw_ds

In [ ]:
bu = raw_ds['ddm_brcs_uncert']
qf = raw_ds['quality_flags']
st = raw_ds['nst_att_status']
fom = raw_ds['prn_fig_of_merit']
les = raw_ds['ddm_les']
rxg = raw_ds['sp_rx_gain']
nsca = raw_ds['nbrcs_scatter_area']
lsca = raw_ds['les_scatter_area']
lat = raw_ds['sp_lat']
lon = raw_ds['sp_lon']
ws = raw_ds['windspeed']

TODO issue with the quality flag

In [ ]:
quality = (bu<1) & (qf == 4) & (st == 0) & (fom > 3) & (rxg > 0) & (les >= 0)

In [ ]:
np.sum((bu<1) & (st==0)).compute()

## Created processed data

In [ ]:
raw_files = prep.open_mfdataset(os.path.join(raw_data_dir, '*.nc'), channels=[0,1,2,3])

In [ ]:
raw_ds = xr.open_mfdataset(os.path.join(raw_data_dir, '*.nc'), engine="netcdf4")

raw_ds

In [ ]:
raw_ds = xr.open_mfdataset([os.path.join(raw_data_dir, dd) for dd in os.listdir(raw_data_dir) if dd.endswith('.nc')])

## Check the new CyGNSS data v3.1

In [ ]:
cygnss_ds

In [ ]:
qf = cygnss_ds.quality_flags[:100].values

In [ ]:
qf == 4

In [ ]:
bu = cygnss_ds.ddm_brcs_uncert[:100].values

In [ ]:
cygnss_ds.ddm_brcs_uncert

In [ ]:
brcs = cygnss_ds.brcs[1].values

In [ ]:
np.sum(brcs, axis=(1,2))

In [ ]:
cygnss_ds.quality_flags[1].values

In [ ]:
plt.imshow(raw_ds.brcs[0].values)
plt.show()
print(raw_ds.windspeed[0].values)